In [1]:
import pandas as pd

base = pd.read_csv('dataset.csv')

In [2]:
import numpy as np

previsores = base.iloc[:1400000,0:20].values
previsores = np.reshape(previsores, (2000, 700, 20))
print(previsores.shape)

classes = base.iloc[:1400000, 20:28].values
classes = np.reshape(classes, (2000, 700, 8))
print(classes.shape)

(2000, 700, 20)
(2000, 700, 8)


In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, CuDNNGRU, Bidirectional, Masking

In [5]:
def criarRede():
    model = Sequential()
  
   # model.add(Masking(mask_value = 0, input_shape = (700, 20)))
  
    model.add(Bidirectional(CuDNNGRU(100, return_sequences = True), input_shape = (700, 20)))
    model.add(Dropout(0.3))
    
    model.add(Bidirectional(CuDNNGRU(100, return_sequences = True)))
    model.add(Dropout(0.3))

    model.add(Bidirectional(CuDNNGRU(100, return_sequences = True)))
    model.add(Dropout(0.3))
    
    model.add(Bidirectional(CuDNNGRU(100, return_sequences = True)))
    model.add(Dropout(0.3))
    
    model.add(Dense(8, activation = 'softmax'))
  
    model.compile(optimizer = 'adam', metrics = ['acc'], loss='categorical_crossentropy')
  
    return model

In [6]:
accuq8 = []
precisionsq8 = []
recallsq8 = []
f1q8 = []

In [7]:
from sklearn.metrics import balanced_accuracy_score, classification_report, precision_score, recall_score, f1_score, confusion_matrix

def train_and_evaluate_model(model, x_train, y_train, x_test, y_test):
    model.fit(x_train, y_train, epochs = 100, verbose = 1, batch_size = 32)
  
    predicted = model.predict(x_test)
  
    y_teste = []
    predict = []
  
    predicted = np.reshape(predicted, (predicted.shape[0] * predicted.shape[1], 8))
    y_test = np.reshape(y_test, (y_test.shape[0] * y_test.shape[1], 8))
    x_test = np.reshape(x_test, (x_test.shape[0] * x_test.shape[1], 20))

    for i in range(len(x_test)):
        cont = 0
        for j in range(len(x_test[i])):
            cont += x_test[i][j]
        if cont != 0:
            y_teste.append(y_test[i])
            predict.append(predicted[i])
    
    y_teste = np.asarray(y_teste)
    predict = np.asarray(predict)

    predicted = predict
    y_test = y_teste

    predicted = np.argmax(predicted, axis=1)
    y_test = np.argmax(y_test, axis=1)

    accuq8.append(balanced_accuracy_score(y_test, predicted)) 
    precisionsq8.append(precision_score(y_test, predicted, average = 'weighted'))
    recallsq8.append(recall_score(y_test, predicted, average = 'weighted'))
    f1q8.append(f1_score(y_test, predicted, average = 'weighted'))
    
    print(classification_report(y_test, predicted))
    print('Acurácia')
    print(balanced_accuracy_score(y_test, predicted))
    print('Precisao')
    print(precision_score(y_test, predicted, average = 'weighted'))
    print('Recall')
    print(recall_score(y_test, predicted, average = 'weighted'))
    print('F1')
    print(f1_score(y_test, predicted, average = 'weighted'))
    print(confusion_matrix(y_test, predicted))

In [8]:
from sklearn.model_selection import KFold

kf = KFold(n_splits = 5, shuffle = True)

for train_index, test_index in kf.split(previsores):
    model = None
    model = criarRede()
    print("TRAIN:", train_index, "TEST:", test_index)
    train_and_evaluate_model(model, previsores[train_index], classes[train_index],
                           previsores[test_index], classes[test_index])

W0822 13:15:31.532517 11676 deprecation.py:506] From c:\users\gabriel\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0822 13:15:31.538491 11676 deprecation.py:506] From c:\users\gabriel\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\ops\init_ops.py:97: calling GlorotUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0822 13:15:31.539533 11676 deprecation.py:506] From c:\users\gabriel\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\ops\init_ops.py

TRAIN: [   0    1    2 ... 1996 1997 1999] TEST: [   3    5   13   29   31   36   42   47   54   61   70   71   75   77
   78   81   82   86   87   88   93   94   98  100  105  118  125  126
  131  144  146  148  151  159  160  171  173  177  180  184  189  190
  191  205  208  211  212  215  218  224  226  230  242  248  252  253
  254  257  260  264  265  274  290  292  296  297  300  306  314  344
  363  366  367  388  390  392  397  399  404  409  411  414  415  416
  420  422  423  428  435  440  441  446  448  450  457  459  469  470
  479  482  487  497  507  509  510  515  520  522  526  532  533  540
  553  555  556  559  573  579  581  590  596  598  601  616  618  622
  636  648  652  653  661  664  675  677  682  684  687  692  693  725
  729  733  735  736  738  739  748  757  759  762  772  775  777  778
  783  786  799  800  806  816  817  819  821  825  836  841  842  847
  848  851  854  855  856  863  869  882  886  895  897  900  901  902
  905  908  911  916  920  9

W0822 13:15:33.197146 11676 deprecation.py:323] From c:\users\gabriel\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/100
1600/1600 [==============================] - 19s 12ms/sample - loss: 0.5862 - acc: 0.1642
Epoch 2/100
1600/1600 [==============================] - 10s 7ms/sample - loss: 0.5358 - acc: 0.1731
Epoch 3/100
1600/1600 [==============================] - 10s 7ms/sample - loss: 0.5236 - acc: 0.1790
Epoch 4/100
1600/1600 [==============================] - 10s 7ms/sample - loss: 0.5163 - acc: 0.1824
Epoch 5/100
1600/1600 [==============================] - 10s 7ms/sample - loss: 0.5132 - acc: 0.1835
Epoch 6/100
1600/1600 [==============================] - 10s 7ms/sample - loss: 0.5096 - acc: 0.1850
Epoch 7/100
1600/1600 [==============================] - 10s 7ms/sample - loss: 0.5056 - acc: 0.1867
Epoch 8/100
1600/1600 [==============================] - 10s 7ms/sample - loss: 0.5008 - acc: 0.1892
Epoch 9/100
1600/1600 [==============================] - 11s 7ms/sample - loss: 0.4983 - acc: 0.1901
Epoch 10/100
1600/1600 [==============================] - 10s 7ms/sample - loss: 0.4927 - 

1600/1600 [==============================] - 10s 7ms/sample - loss: 0.3102 - acc: 0.2564
Epoch 82/100
1600/1600 [==============================] - 10s 6ms/sample - loss: 0.3098 - acc: 0.2562
Epoch 83/100
1600/1600 [==============================] - 10s 7ms/sample - loss: 0.3079 - acc: 0.2570
Epoch 84/100
1600/1600 [==============================] - 11s 7ms/sample - loss: 0.3074 - acc: 0.2571
Epoch 85/100
1600/1600 [==============================] - 11s 7ms/sample - loss: 0.3054 - acc: 0.2578
Epoch 86/100
1600/1600 [==============================] - 11s 7ms/sample - loss: 0.3048 - acc: 0.2579
Epoch 87/100
1600/1600 [==============================] - 11s 7ms/sample - loss: 0.3040 - acc: 0.2582
Epoch 88/100
1600/1600 [==============================] - 11s 7ms/sample - loss: 0.3028 - acc: 0.2591
Epoch 89/100
1600/1600 [==============================] - 11s 7ms/sample - loss: 0.3018 - acc: 0.2589
Epoch 90/100
1600/1600 [==============================] - 11s 7ms/sample - loss: 0.3005 - acc: 

...

1600/1600 [==============================] - 10s 6ms/sample - loss: 0.4933 - acc: 0.1910
Epoch 10/100
1600/1600 [==============================] - 10s 6ms/sample - loss: 0.4915 - acc: 0.1916
Epoch 11/100
1600/1600 [==============================] - 10s 6ms/sample - loss: 0.4885 - acc: 0.1931
Epoch 12/100
1600/1600 [==============================] - 10s 6ms/sample - loss: 0.4842 - acc: 0.1948
Epoch 13/100
1600/1600 [==============================] - 10s 6ms/sample - loss: 0.4850 - acc: 0.1943
Epoch 14/100
1600/1600 [==============================] - 10s 6ms/sample - loss: 0.4802 - acc: 0.1965
Epoch 15/100
1600/1600 [==============================] - 10s 6ms/sample - loss: 0.4743 - acc: 0.1989
Epoch 16/100
1600/1600 [==============================] - 10s 6ms/sample - loss: 0.4729 - acc: 0.1995
Epoch 17/100
1600/1600 [==============================] - 10s 6ms/sample - loss: 0.4700 - acc: 0.2007
Epoch 18/100
1600/1600 [==============================] - 10s 6ms/sample - loss: 0.4661 - acc: 

1600/1600 [==============================] - 10s 6ms/sample - loss: 0.2995 - acc: 0.2584
Epoch 90/100
1600/1600 [==============================] - 10s 6ms/sample - loss: 0.3009 - acc: 0.2580
Epoch 91/100
1600/1600 [==============================] - 10s 6ms/sample - loss: 0.2983 - acc: 0.2587
Epoch 92/100
1600/1600 [==============================] - 10s 6ms/sample - loss: 0.2981 - acc: 0.2590
Epoch 93/100
1600/1600 [==============================] - 10s 6ms/sample - loss: 0.2966 - acc: 0.2594
Epoch 94/100
1600/1600 [==============================] - 10s 6ms/sample - loss: 0.2954 - acc: 0.2597
Epoch 95/100
1600/1600 [==============================] - 10s 6ms/sample - loss: 0.2951 - acc: 0.2599
Epoch 96/100
1600/1600 [==============================] - 10s 6ms/sample - loss: 0.2941 - acc: 0.2601
Epoch 97/100
1600/1600 [==============================] - 10s 6ms/sample - loss: 0.2934 - acc: 0.2607
Epoch 98/100
1600/1600 [==============================] - 10s 6ms/sample - loss: 0.2930 - acc: 

c:\users\gabriel\appdata\local\programs\python\python36\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\gabriel\appdata\local\programs\python\python36\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\gabriel\appdata\local\programs\python\python36\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1207
           1       0.64      0.71      0.67     22248
           2       0.47      0.31      0.37      4192
           3       0.76      0.82      0.79     32085
           4       0.60      0.28      0.38       624
           5       0.48      0.56      0.51     20814
           6       0.38      0.13      0.20      9138
           7       0.41      0.43      0.42     11383

    accuracy                           0.60    101691
   macro avg       0.47      0.40      0.42    101691
weighted avg       0.58      0.60      0.58    101691

Acurácia
0.40446799989857135
Precisao
0.5799384885957153
Recall
0.6010659743733467
F1
0.5828036386427987
[[    0   297    20   113     2   597    62   116]
 [    0 15701   142  1873     6  3312   276   938]
 [    0   464  1306   842     2   811   105   662]
 [    0  1789   370 26185    63  2029   183  1466]
 [    0    69     4   279   173    55   

1600/1600 [==============================] - 10s 6ms/sample - loss: 0.3596 - acc: 0.2388
Epoch 50/100
1600/1600 [==============================] - 10s 6ms/sample - loss: 0.3583 - acc: 0.2392
Epoch 51/100
1600/1600 [==============================] - 10s 6ms/sample - loss: 0.3551 - acc: 0.2404
Epoch 52/100
1600/1600 [==============================] - 10s 6ms/sample - loss: 0.3530 - acc: 0.2410
Epoch 53/100
1600/1600 [==============================] - 10s 6ms/sample - loss: 0.3513 - acc: 0.2414
Epoch 54/100
1600/1600 [==============================] - 10s 6ms/sample - loss: 0.3499 - acc: 0.2422
Epoch 55/100
1600/1600 [==============================] - 10s 6ms/sample - loss: 0.3477 - acc: 0.2425
Epoch 56/100
1600/1600 [==============================] - 10s 6ms/sample - loss: 0.3467 - acc: 0.2431
Epoch 57/100
1600/1600 [==============================] - 10s 6ms/sample - loss: 0.3428 - acc: 0.2439
Epoch 58/100
1600/1600 [==============================] - 10s 6ms/sample - loss: 0.3416 - acc: 

In [9]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_16 (Bidirectio (None, 700, 200)          73200     
_________________________________________________________________
dropout_16 (Dropout)         (None, 700, 200)          0         
_________________________________________________________________
bidirectional_17 (Bidirectio (None, 700, 200)          181200    
_________________________________________________________________
dropout_17 (Dropout)         (None, 700, 200)          0         
_________________________________________________________________
bidirectional_18 (Bidirectio (None, 700, 200)          181200    
_________________________________________________________________
dropout_18 (Dropout)         (None, 700, 200)          0         
_________________________________________________________________
bidirectional_19 (Bidirectio (None, 700, 200)         

In [11]:
print('Acurácias total')
print(accuq8)
a = np.array(accuq8)
print(a.mean())
print('Precision total')
print(precisionsq8)
p = np.array(precisionsq8)
print(p.mean())
print('Recalls total')
print(recallsq8)
r = np.array(recallsq8)
print(r.mean())
print('F1 total')
print(f1q8)
f = np.array(f1q8)
print(f.mean())

Acurácias total
[0.3808415176149993, 0.4011994586249975, 0.3854811009932829, 0.40446799989857135, 0.39296823991235785]
0.3929916634088418
Precision total
[0.5843429913694231, 0.577430473407783, 0.5719754768091974, 0.5799384885957153, 0.578623710713331]
0.5784622281790901
Recalls total
[0.5984322642412637, 0.5941223193010325, 0.5876692174588928, 0.6010659743733467, 0.5986255517047306]
0.5959830654158532
F1 total
[0.5779087520420751, 0.5752851601719448, 0.5689934762513187, 0.5828036386427987, 0.5780177462533034]
0.5766017546722881
